In [ ]:
import time
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use('fivethirtyeight')

from nfl_pickem import Pickem

In [ ]:
pk = Pickem()
print(pk.file_path)
pk.pull_data()
df = pk.build_schedule()

In [ ]:
team = 'NE'
cond = df.team == team
plt.plot(df[cond].week.values,
         df[cond].win_prob.values - df[cond].elo_prob1.values,
         label=team+' Win Prob')
plt.scatter(df[cond&(df.result1 > 0.9)].week.values,
            0.3*df[cond&(df.result1 > 0.9)].result1.values - 0.15,
            label='Win', c='C3')
plt.scatter(df[cond&(df.result1 < 0.9)].week.values,
            0.3*df[cond&(df.result1 < 0.9)].result1.values - 0.15,
            label='Lose/Tie', c='C2')
plt.ylabel('Win Prob Difference (from Week 1 ELO)')
plt.xlabel('Week')
plt.xticks(df[cond].week.values,
           df[cond].week.values)
plt.legend(loc='lower right')
plt.show()

## Historical Validation

In [ ]:
df = pd.read_csv('results/strategy_analysis.csv')
print(df.shape)

In [ ]:
df = df.sort_values(by=['season', 'max_week'])
for season in df.season.unique():
    cond = df.season == season
    plt.plot(df[cond].max_week.values,
             df[cond].elim_week.values,
             label=season)
plt.legend()
plt.show()

In [ ]:
ew = df.groupby('max_week')['elim_week'].agg(['mean', 'std']).reset_index()
plt.plot(ew.max_week.values,
         ew['mean'].values)
plt.errorbar(ew.max_week.values,
             ew['mean'].values,
             yerr=ew['std'].values,
             elinewidth=2)
plt.xticks(ew.max_week.values,
           ew.max_week.values)
plt.xlabel('Max Forecast Distance')
plt.ylabel('Expected Elimination Week')
plt.title('Pool Elimination (1997-2017)')
plt.show()

In [ ]:
ew = df.groupby('max_week')['correct'].agg(['mean', 'std']).reset_index()
plt.plot(ew.max_week.values,
         ew['mean'].values)
plt.errorbar(ew.max_week.values,
             ew['mean'].values,
             yerr=ew['std'].values,
             elinewidth=2)
plt.xticks(ew.max_week.values,
           ew.max_week.values)
plt.xlabel('Max Forecast Distance')
plt.ylabel('Expected Correct Picks')
plt.title('Pick Success (1997-2017)')
plt.show()

## Checking New Model

In [ ]:
pk = Pickem()
pk.file_path = '../nfl-pickem/data/nfl_elo.csv'
pk.pull_data()
pk.data_.head()

In [ ]:
df = pk.build_schedule(season=2017,
                       qb_elo_model=True)
cols = ['neutral', 'team1', 'team2', 'elo1_pre', 'elo2_pre',
        'qbelo1_pre', 'qbelo2_pre',
        'qb1_value_pre', 'qb2_value_pre', 'qb1_adj', 'qb2_adj',
        'qbelo_prob1', 'qbelo_prob2']
print(1/(1+np.power(10, -1*(df.qbelo2_pre.iloc[8]-df.qbelo1_pre.iloc[8]
                 +df.qb2_adj.iloc[8]-df.qb1_adj.iloc[8]-55-4*1633/1000
                   )/400)))
display(df[(df.team1 == 'GB')|(df.team2 == 'GB')][cols])

In [ ]:
df.columns

In [ ]:
68*3.3

In [ ]:
cols = ['date', 'neutral', 'team1', 'team2', 'elo1_pre',
        'elo2_pre', 'qbelo1_pre', 'qbelo2_pre',
        'qb1_value_pre', 'qb2_value_pre', 'qb1_adj', 'qb2_adj',
        'qbelo_prob1', 'qbelo_prob2', 'qbelo1_post', 'qbelo2_post',]
df[(df.season == 2019)&
   ((df.team1 == 'GB')|(df.team2 == 'GB'))][cols].head(10)

In [ ]:
1/(1+10**(-1*(1588.897931+55-1455.13137+174-191+4*185.5/1000)/400))

In [ ]:
df.columns

In [ ]:
(-400*np.log10(1/df.qbelo_prob1.iloc[15751]-1)-(1592.676645+55-1588.11685+18.210896+2.857484))*1000/4

In [ ]:
2581*4/1000

In [ ]:
df.qbelo_prob1.iloc[15751]

In [ ]:
tmp = df[(df.season == 2017)]
tmp['elo_adj'] = -400*np.log10(1/df.elo_prob1 - 1) - (tmp['elo1_pre'] - tmp['elo2_pre'])
tmp[['neutral', 'elo_adj']]